In [ ]:
#  > /dev/null for silent execution (without logs)
!apt-get update  > /dev/null
!apt install chromium-chromedriver > /dev/null

In [ ]:
!pip3 install selenium > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Bypass captcha (recognizing bot)
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("window-size=1280,800")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

driver = webdriver.Chrome('chromedriver',options=options)

In [ ]:
def get_url_from_context(context):
  f = open('/content/sample_data/urls.json')
  return json.load(f)[context]

In [ ]:
def pd_drop_columns(df, cols):
  for col in cols:
    df.drop(col, inplace=True, axis=1)
  return df

In [ ]:
def get_fiis_base_info():
  driver.get(get_url_from_context('url_fii_base'))
  main_table = driver.find_element(by=By.CLASS_NAME, value='tabela_principal')

  # outerHTML to get table tag too
  df_main_table = pd.read_html(main_table.get_attribute('outerHTML'))[0]
  df_main_table = pd_drop_columns(df_main_table, ['RELATÓRIOS DE ANÁLISE','FEED'])
  df_main_table["VALOR COTA"] = df_main_table["VALOR COTA"].apply(lambda x: str(x).split(' ')[0])

  return df_main_table

In [ ]:
def get_ticker_info(ticker):
  try:
    page = requests.get(get_url_from_context('url_fii_info')+ticker)
    soup = BeautifulSoup(page.content, 'html.parser')

    infos = soup.find_all(class_='info')

    ticker_price  = infos[0].find('strong', {'class': 'value'}).text
    last_dividend = infos[15].find('strong', {'class': 'value'}).text
    dy            = infos[15].find('b', {'class': 'sub-value'}).text
    dy_12_months  = infos[3].find('strong', {'class': 'value'}).text
    p_vp          = infos[6].find('strong', {'class': 'value'}).text

    ret = pd.DataFrame({
            'CÓDIGO': [ticker],
            'ULTIMO_DIVIDENDO': [last_dividend],
            'DY': [dy],
            'DY12': [dy_12_months],
            'P/VP': [p_vp]
          })
    return ret
  except Exception as e:
    # print(e)
    ret = pd.DataFrame({
            'CÓDIGO': [None],
            'ULTIMO_DIVIDENDO': [None],
            'DY': [None],
            'DY12': [None],
            'P/VP': [None]
          })
    return ret

In [ ]:
def get_fiis_full(df_main_table):
  ticker_infos = None
  for index, row in df_main_table.iterrows():
    ticker = row['CÓDIGO']
    if ticker_infos is None:
      ticker_infos = get_ticker_info(ticker)
    else:
      ticker_infos = pd.concat([ticker_infos, get_ticker_info(ticker)], ignore_index=True)
  ticker_infos.dropna(how='all', inplace=True)

  return pd.merge(df_main_table, ticker_infos, how='left', on='CÓDIGO')

In [ ]:
def get_wallet():
  df_wallet = pd_drop_columns(pd.read_csv('/content/Historico.csv', sep=';', decimal=','), ['Data', 'Total'])
  df_wallet = df_wallet.astype({'Valor':'float','Quantidade':'int'})

  df_wallet["VALOR_TOTAL"] = df_wallet["Valor"] * df_wallet["Quantidade"]
  df_wallet = df_wallet.groupby(['Ticker'], as_index=False).agg(VALOR_TOTAL=('VALOR_TOTAL', 'sum'), QUANTIDADE=('Quantidade', 'sum')).round(2)
  df_wallet["PM"] = (df_wallet["VALOR_TOTAL"] / df_wallet["QUANTIDADE"]).round(2)

  return df_wallet

In [ ]:
df_fiis_full = get_fiis_full(get_fiis_base_info())

display(df_fiis_full)

,CÓDIGO,NOME,VALOR COTA,DATA IPO,VALOR IPO,SEGMENTO,ADMINISTRADOR,ULTIMO_DIVIDENDO,DY,DY12,P/VP
0,ABCP11,Grand Plaza Shopping,"72,50",12/03/2004,"R$18,50",Shopping/Varejo,RIO BRAVO,"0,5900","0,8138","8,87","0,79"
1,AFHI11,AF Invest CRI,"102,79",05/04/2021,"R$100,00",Recebíveis Imobiliários,BTG PACTUAL,"1,3500","1,3527","13,13","1,07"
2,AFOF11,Alianza FOF,"91,00",23/03/2021,"R$100,00",Fundo de Fundos,BRL TRUST,"1,1355","1,2697","13,48","0,89"
3,AIEC11,Autonomy Edifícios Corporativos,"77,10",02/09/2020,"R$100,00",Lajes Comerciais,MAF,"0,7200","0,8967","11,50","0,76"
4,ALMI11,Torre Almirante,"930,00",01/06/2005,"R$1.000,00",Lajes Comerciais,BTG PACTUAL,"0,3711","0,0217","0,00","0,42"
...,...,...,...,...,...,...,...,...,...,...,...
429,XPPR11,XP Properties,"55,15",09/12/2019,"R$100,00",Lajes Comerciais,VÓRTX,"0,4500","0,7692","10,90","0,70"
430,XPSF11,XP Selection,"7,23",17/02/2019,"R$10,00",Fundo de Fundos,XP INVESTIMENTOS,"0,0680","0,0926","11,45","0,08"
431,XTED11,TRX Edifícios Corporativos,"6,02",13/11/2012,"R$100,00",Lajes Comerciais,BTG PACTUAL,"0,3233","1,4439","0,00","0,43"
432,YUFI11,Yuca,"105,00",09/11/2020,"R$100,00",Residencial,BRL TRUST,"0,2044",-,"0,00",-


In [ ]:
df_wallet = get_wallet()

display(df_wallet)

In [ ]:
columns_to_drop = ['Ticker', 'DATA IPO', 'VALOR IPO', 'DY12', 'DY']
df_indicator = pd_drop_columns(pd.merge(df_wallet, df_fiis_full, left_on='Ticker', right_on='CÓDIGO'), columns_to_drop)

df_indicator["ULTIMO_DIVIDENDO"] = df_indicator.apply(lambda x: float(x["ULTIMO_DIVIDENDO"].replace('.', '').replace(',', '.')),axis=1)
df_indicator["VALOR COTA"] = df_indicator.apply(lambda x: float(x["VALOR COTA"].replace('.', '').replace(',', '.')),axis=1)

df_indicator["DY_CARTEIRA"] = df_indicator["ULTIMO_DIVIDENDO"] / df_indicator["PM"] * 100
df_indicator["DY_FII_ATUAL"] = df_indicator["ULTIMO_DIVIDENDO"] / df_indicator["VALOR COTA"] * 100
display(df_indicator)

In [ ]:
print(df_indicator['VALOR_TOTAL'].sum())

In [ ]:
print(df_indicator['DY'].sum() / len(df_indicator))